In [1]:
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, DepthwiseConv2D, SeparableConv2D
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D
from keras.layers import BatchNormalization, concatenate, add, Dropout, ReLU, Lambda, Activation, LeakyReLU

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from time import time
import numpy as np

Using TensorFlow backend.


In [2]:
def alexnet(input_shape, n_classes):
  input = Input(input_shape)
  
  # actually batch normalization didn't exist back then
  # they used LRN (Local Response Normalization) for regularization
  x = Conv2D(96, 11, strides=4, padding='same', activation='relu')(input)
  x = BatchNormalization()(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = Conv2D(256, 5, padding='same', activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
  x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
  x = Conv2D(256, 3, strides=1, padding='same', activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = Flatten()(x)
  x = Dense(4096, activation='relu')(x)
  x = Dense(4096, activation='relu')(x)
  
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  return model

In [3]:
input_shape = 224, 224, 3
n_classes = 1000

K.clear_session()
model = alexnet(input_shape, n_classes)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 96)        34944     
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 256)       0         
__________

In [ ]:
repetitions = 10
input = np.random.randn(1, *input_shape)

output = model.predict(input)
start = time()
for _ in range(repetitions):
  output = model.predict(input)
  
print((time() - start) / repetitions)

In [ ]:
###########VGG

In [2]:
def vgg(input_shape, n_classes):
  filters = 64, 128, 256, 512, 512
  repetitions = 2, 2, 3, 3, 3
  
  def vgg_block(x, f, r):
    for _ in range(r):
      x = Conv2D(f, 3, padding='same', activation='relu')(x)
    x = MaxPool2D(2, strides=2)(x)
    return x
  
  
  input = Input(input_shape)
  
  x = input
  for f, r in zip(filters, repetitions):
    x = vgg_block(x, f, r)
  
  x = Flatten()(x)
  x = Dense(4096, activation='relu')(x)
  x = Dense(4096, activation='relu')(x)
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  return model

In [3]:
input_shape = 224, 224, 3
n_classes = 1000

K.clear_session()
model = vgg(input_shape, n_classes)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
__________

In [ ]:
repetitions = 10
input = np.random.randn(1, *input_shape)

output = model.predict(input)
start = time()
for _ in range(repetitions):
  output = model.predict(input)
  
print((time() - start) / repetitions)

In [ ]:
#############Inception

In [3]:
def googlenet(input_shape, n_classes):
  
  def inception_block(x, f):
    t1 = Conv2D(f[0], 1, activation='relu')(x)
    
    t2 = Conv2D(f[1], 1, activation='relu')(x)
    t2 = Conv2D(f[2], 3, padding='same', activation='relu')(t2)
    
    t3 = Conv2D(f[3], 1, activation='relu')(x)
    t3 = Conv2D(f[4], 5, padding='same', activation='relu')(t3)
    
    t4 = MaxPool2D(3, 1, padding='same')(x)
    t4 = Conv2D(f[5], 1, activation='relu')(t4)
    
    output = concatenate([t1, t2, t3, t4])
    return output
  
  
  input = Input(input_shape)
  
  x = Conv2D(64, 7, strides=2, padding='same', activation='relu')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = Conv2D(64, 1, activation='relu')(x)
  x = Conv2D(192, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = inception_block(x, [64, 96, 128, 16, 32, 32])
  x = inception_block(x, [128, 128, 192, 32, 96, 64])
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = inception_block(x, [192, 96, 208, 16, 48, 64])
  x = inception_block(x, [160, 112, 224, 24, 64, 64])
  x = inception_block(x, [128, 128, 256, 24, 64, 64])
  x = inception_block(x, [112, 144, 288, 32, 64, 64])
  x = inception_block(x, [256, 160, 32, 32, 128, 128])
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = inception_block(x, [256, 160, 320, 32, 128, 128])
  x = inception_block(x, [384, 192, 384, 48, 128, 128])
  
  x = AvgPool2D(7, strides=1)(x)
  x = Dropout(0.4)(x)
  
  x = Flatten()(x)
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  return model

In [7]:
input_shape = 224, 224, 3
n_classes = 1000

K.clear_session()
model = googlenet(input_shape, n_classes)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 56, 56, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 56, 56, 64)   4160        max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_3 (

In [5]:
repetitions = 10
input = np.random.randn(1, *input_shape)

output = model.predict(input)
start = time()
for _ in range(repetitions):
  output = model.predict(input)
  
print((time() - start) / repetitions)

0.010972976684570312


In [8]:
#####################ResNet

In [11]:
def resnet(input_shape, n_classes):
  
  def conv_bn_rl(x, f, k=1, s=1, p='same'):
    x = Conv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
  
  
  def conv_block(tensor, f1, f2, s):
    x = conv_bn_rl(tensor, f1)
    x = conv_bn_rl(x, f1, 3, s=s)
    x = Conv2D(f2, 1)(x)
    x = BatchNormalization()(x)
    
    shortcut = Conv2D(f2, 1, strides=s, padding='same')(tensor)
    shortcut = BatchNormalization()(shortcut)
    
    x = add([shortcut, x])
    output = ReLU()(x)
    
    return output
  
  
  def identity_block(tensor, f1, f2):
    x = conv_bn_rl(tensor, f1)
    x = conv_bn_rl(x, f1, 3)
    x = Conv2D(f2, 1)(x)
    x = BatchNormalization()(x)
    
    x = add([tensor, x])
    output = ReLU()(x)
    
    return output
  
  
  def resnet_block(x, f1, f2, r, s=2):
    x = conv_block(x, f1, f2, s)
    
    for _ in range(r-1):
      x = identity_block(x, f1, f2)
    
    return x
  
  
  input = Input(input_shape)
  
  x = conv_bn_rl(input, 64, 7, 2)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = resnet_block(x, 64, 256, 3, 1)
  x = resnet_block(x, 128, 512, 4)
  x = resnet_block(x, 256, 1024, 6)
  x = resnet_block(x, 512, 2048, 3)
  
  x = AvgPool2D(7, strides=1)(x)
  x = Flatten()(x)
  
  output = Dense(n_classes, activation='softmax')(x)
  model = Model(input, output)
  
  return model

In [10]:
input_shape = 224, 224, 3
n_classes = 1000

K.clear_session()
model = resnet(input_shape, n_classes)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 112, 112, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [13]:
repetitions = 10
input = np.random.randn(1, *input_shape)

output = model.predict(input)
start = time()
for _ in range(repetitions):
  output = model.predict(input)
  
print((time() - start) / repetitions)

0.01266617774963379


In [2]:
##############DenseNet

In [3]:
def densenet(input_shape, n_classes, k=32):
  repetitions = 6, 12, 48, 32
  
  def bn_rl_conv(x, f, k=1, s=1, p='same'):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(f, k, strides=s, padding=p)(x)
    return x
  
  
  def dense_block(tensor, k):
    x = bn_rl_conv(tensor, 4*k)
    x = bn_rl_conv(x, k, 3)
    output = concatenate([tensor, x])
    return output
  
  
  def transition(x):
    x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)
    x = AvgPool2D(2, strides=2, padding='same')(x)
    return x
  
  
  def dense_blocks(x, r, k):
    for _ in range(r):
      x = dense_block(x, k)
    return x
  
  
  input = Input(input_shape)
  
  x = Conv2D(64, 7, strides=2, padding='same')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  for r in repetitions:
    d = dense_blocks(x, r, k)
    x = transition(d)
  
  x = AvgPool2D(7, strides=1)(d)
  x = Flatten()(x)
  
  output = Dense(n_classes, activation='softmax')(x)
  model = Model(input, output)
  
  return model

In [6]:
input_shape = 224, 224, 3
n_classes = 1000

K.clear_session()
model = densenet(input_shape, n_classes)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 56, 56, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 56, 56, 64)   256         max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
re_lu_1 (R

__________________________________________________________________________________________________
batch_normalization_194 (BatchN (None, 7, 7, 1824)   7296        concatenate_95[0][0]             
__________________________________________________________________________________________________
re_lu_194 (ReLU)                (None, 7, 7, 1824)   0           batch_normalization_194[0][0]    
__________________________________________________________________________________________________
conv2d_195 (Conv2D)             (None, 7, 7, 128)    233600      re_lu_194[0][0]                  
__________________________________________________________________________________________________
batch_normalization_195 (BatchN (None, 7, 7, 128)    512         conv2d_195[0][0]                 
__________________________________________________________________________________________________
re_lu_195 (ReLU)                (None, 7, 7, 128)    0           batch_normalization_195[0][0]    
__________

In [7]:
repetitions = 10
input = np.random.randn(1, *input_shape)

output = model.predict(input)
start = time()
for _ in range(repetitions):
  output = model.predict(input)
  
print((time() - start) / repetitions)

0.06028313636779785


In [2]:
##############U-Net

In [3]:
def unet(input_shape=(572, 572, 1), f=64, steps=4, n_classes=2):
  
  def downstream(x, f):
    x = Conv2D(f, 3, activation='relu')(x)
    d = Conv2D(f, 3, activation='relu')(x)
    x = MaxPool2D(2, strides=2, padding='same')(d)
    return d, x
  
  
  def crop_merge(x, d):
    _, xw, xh, _ = K.int_shape(x)
    _, dw, dh, _ = K.int_shape(d)
    mw, mh = (dw-xw)//2, (dh-xh)//2
    
    d = Lambda(lambda x: x[:, mw: dw-mw, mh: dh-mh, :])(d)
    x = concatenate([d, x])
    return x
    
  
  def upstream(x, f, d):
    x = UpSampling2D()(x)
    x = Conv2D(f, 2, padding='same')(x)
    x = crop_merge(x, d)
    x = Conv2D(f, 3, activation='relu')(x)
    x = Conv2D(f, 3, activation='relu')(x)
    return x
    
  
  input = Input(input_shape)
  x = input
  
  downsampled = []
  for i in range(steps+1):
    d, x = downstream(x, f*2**i)
    downsampled.append(d)
  x = downsampled.pop()
  
  for i in range(steps-1, -1, -1):
    x = upstream(x, f*2**i, downsampled[i])
  
  output = Conv2D(n_classes, 1)(x)
  model = Model(input, output)
  
  return model

In [6]:
input_shape = 572, 572, 1
n_classes = 1000

K.clear_session()
model = unet(input_shape, n_classes)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 572, 572, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 570, 570, 100 10000       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 568, 568, 100 9001000     conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 284, 284, 100 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [ ]:
repetitions = 1
input = np.random.randn(1, *input_shape)

output = model.predict(input)
start = time()
for _ in range(repetitions):
  output = model.predict(input)
  
print((time() - start) / repetitions)